In [ ]:
import pyrebase
import pandas as pd
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.base import MIMEBase
from io import StringIO
import smtplib
from email import encoders
from flask import *
from flask_cors import CORS
from datetime import date
from datetime import datetime
import cv2
import numpy as np
from PIL import Image, ImageDraw, ImageFont

config = {
    "apiKey": "",
    "authDomain": "",
    "databaseURL": "",
    "projectId": "",
    "storageBucket": "",
    "messagingSenderId": "",
    "appId": "",
    "measurementId": ""
}

firebase = pyrebase.initialize_app(config)

db = firebase.database()

#db.child("names").push({"name": "Haider"})

#db.child("names").child("name").update({"name": "batth"})

#users = db.child("names").child("name").get()
#print(users.key())

#db.child("names").remove()



app = Flask(_name_)

@app.route("/attendance", methods = ['POST'])
def attendance():
    data = request.json
    reg = data['reg']
    name = data['name']
    team = data['teamName']
    check = True

    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")

    db.child(team).child(date.today()).child(reg).set({"name": name, "time": current_time})

    if check:
        return jsonify({"code": 62})
    else:
        return jsonify({"code": 26})

@app.route("/getAttendance", methods = ['POST'])
def getAttendance():
    data = request.json
    teamName = data['TeamName']
    email = data['Email']
    check = True

    users = db.child(teamName).get()
    json = users.val()

    df = pd.DataFrame(json)

    df = pd.DataFrame(columns = ["RegistrationNumber", "Name"])

    for i in json:
        for k, j in json[i].items():
            df_ = pd.DataFrame({'RegistrationNumber': k, 'Name':j['name']}, index = [0])
            df = pd.concat([df, df_], ignore_index = True)

    df = df.drop_duplicates(subset = 'RegistrationNumber', keep = "first")

    for i in json:
        df[i] = "Absent"
        for k, j in json[i].items():
            df.loc[df.RegistrationNumber == k, i] = j['time']

    df = df.sort_values(by=['RegistrationNumber'])#, ascending = False)

    textStream = StringIO()
    df.to_csv(textStream, index=False)

    #Email
    # Create a multipart message
    msg = MIMEMultipart()
    body_part = MIMEText("", 'plain')
    msg['Subject'] = "Attendance " + teamName
    msg['From'] = "haiderirfan62test@gmail.com"
    msg['To'] = email

    # Add body to email
    msg.attach(body_part)

    msg.attach(MIMEApplication(textStream.getvalue(), Name= teamName + ".csv"))

    try:
        # Create SMTP object
        smtp_port = 587
        smtp_server = "smtp.gmail.com"
        smtp_obj = smtplib.SMTP(smtp_server, smtp_port)

        # Login to the server
        smtp_obj.ehlo()
        smtp_obj.starttls()
        smtp_obj.ehlo()
        smtp_obj.login("haiderirfan62test@gmail.com", "dauspfglexmrwdtn")

        # Convert the message to a string and send it
        smtp_obj.sendmail(msg['From'], msg['To'], msg.as_string())
    except Exception as e:
        print(e)
    finally:
        smtp_obj.quit()

    if check:
        return jsonify({"code": 62})
    else:
        return jsonify({"code": 26})

@app.route("/getCertificate", methods = ['GET', 'POST'])
def getCertificate():
    data = request.json
    name = data['name']
    titleName = data['title']
    hostName = data['hostName']
    email = data['email']
    #email = "haiderirfan62batth@gmail.com"
    check = True

    # Load the input image
    img = cv2.imread("certificate.png")

    # Convert the input image from BGR to RGB
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Define the text to be written on the image
    text = name

    # Create an Image object from the input image
    img_pil = Image.fromarray(img)

    # Load the desired font
    font = ImageFont.truetype("Segoe UI.ttf", 67)

    # Get the size of the text
    text_size = 67

    # Create a Draw object
    draw = ImageDraw.Draw(img_pil)

    # Define the text location on the image
    text_location = (95, 410)

    # Define the color of the text
    color = (0, 118, 213)

    # Write the text on the image
    draw.text(text_location, text, font=font, fill=color)

    # Define the text to be written on the image
    title = titleName
    text = """In recognition of your attendance and completion of the\nMicrosoft Student Ambassadors \n""" + title + """ workshop."""

    # Load the desired font
    font = ImageFont.truetype("Segoe UI.ttf", 40)

    # Get the size of the text
    text_size = 40

    # Create a Draw object
    draw = ImageDraw.Draw(img_pil)

    # Define the text location on the image
    text_location = (95, 600)

    # Define the color of the text
    color = (0, 0, 0)

    # Write the text on the image
    draw.text(text_location, text, font=font, fill=color)

    host = hostName

    # Define the text to be written on the image
    text = """Event Hosted By\n""" + host + """\nMicrosoft Learn Student Ambassador"""

    # Load the desired font
    font = ImageFont.truetype("Segoe UI.ttf", 32)

    # Get the size of the text
    text_size = 32

    # Create a Draw object
    draw = ImageDraw.Draw(img_pil)

    # Define the text location on the image
    text_location = (95, 850)

    # Define the color of the text
    color = (0, 0, 0)

    # Write the text on the image
    draw.text(text_location, text, font=font, fill=color)


    # Convert the image back to a numpy array
    img = np.array(img_pil)

    # Convert the image back to BGR
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

    # Save the output image
    cv2.imwrite("certificate1.jpeg", img)
    print("Noice")

    #Email
    # Create a multipart message
    msg = MIMEMultipart()
    body_part = MIMEText('Hey there!\nYour E-Certificate is attached for the "' + title + '" workshop you attended.\n\n\nRegards,\n' + hostName + '\nLead Microsoft Learn Student Ambassador', 'plain')
    msg['Subject'] = "E-Certificate"
    msg['From'] = "haiderirfan62test@gmail.com"
    msg['To'] = email

    # Add body to email
    msg.attach(body_part)

    with open("certificate1.jpeg", "rb") as f:
        img = MIMEImage(f.read())

    msg.attach(img)

    try:
        # Create SMTP object
        smtp_port = 587
        smtp_server = "smtp.gmail.com"
        smtp_obj = smtplib.SMTP(smtp_server, smtp_port)

        # Login to the server
        smtp_obj.ehlo()
        smtp_obj.starttls()
        smtp_obj.ehlo()
        smtp_obj.login("haiderirfan62test@gmail.com", "dauspfglexmrwdtn")

        # Convert the message to a string and send it
        smtp_obj.sendmail(msg['From'], msg['To'], msg.as_string())
    except Exception as e:
        print(e)
    finally:
        smtp_obj.quit()

    if check:
        return jsonify({"code": 62})
    else:
        return jsonify({"code": 26})

@app.route('/data')
def data():
    team = 'Automation'
    users = db.child(team).get()

    return users.val()


@app.route('/', methods = ['GET', 'POST'])
def basic():
    return "Hello World"

if _name_ == "_main_":
    app.run(host='0.0.0.0', port='80')